# CNN Factorio AI

In [1]:
import torch

print(torch.version.cuda)
# Check if CUDA GPU is available
if torch.cuda.is_available():
    print("CUDA is available. Training on GPU.")
else:
    print("CUDA not available. Training on CPU.")


12.8
CUDA is available. Training on GPU.


### Data processing

In [8]:
import numpy as np

def str_to_array(s):
    """Convert a string representation of a board to a numpy array."""
    lines = s.split("\n")[:-1]  # Remove the last empty line if it exists
    array = []
    for y, line in enumerate(lines):
        array.append([])
        for cell in line.split():
            array[y].append(int(cell))
    return np.array(array, dtype=np.int32)

# Parse the data
def parse_data(file):
    inputs = []
    outputs = []

    index_board = ""
    with open(file,"r") as f:
        for i,line in enumerate(f):
            line = line.strip()
            if line.startswith("reward="):
                if index_board != "":
                    inputs.append(str_to_array(index_board))
                    outputs.append(float(reward))
                    index_board = ""
                reward = line.split("=")[1]
            else:
                index_board += line + "\n"
        inputs.append(str_to_array(index_board))
        outputs.append(float(reward))
    return np.array(inputs), np.array(outputs).reshape(-1, 1)

X, y = parse_data("game.txt")
print(X.shape, y.shape)

(100000, 5, 5) (100000, 1)


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from var import CELL_NUMBER

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_flat = X.reshape((X.shape[0], -1))

X_train , X_test, y_train,y_test = train_test_split(X_flat, y,test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_scaled = scaler_X.fit_transform(X_train).reshape(-1, CELL_NUMBER, CELL_NUMBER)
y_train_scaled = scaler_y.fit_transform(y_train)

X_test_scaled = scaler_X.transform(X_test).reshape(-1, CELL_NUMBER, CELL_NUMBER)
y_test_scaled = scaler_y.transform(y_test)

(80000, 25) (80000, 1) (20000, 25) (20000, 1)


In [21]:
from torch.utils.data import Dataset, DataLoader

class GridRewardDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1, 5, 5)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# DataLoader
ds_train = GridRewardDataset(X_train_scaled, y_train_scaled)
ds_test = GridRewardDataset(X_test_scaled, y_test_scaled)

train = DataLoader(ds_train, batch_size=32, shuffle=True,num_workers=27)
test = DataLoader(ds_test, batch_size=32,num_workers=27)

### Lightning Module

In [25]:
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint


#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#os.environ['CUDA_LAUNCH_BLOCKING']='1'

class RewardCNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),   # (B, 32, 5, 5)
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),  # (B, 64, 5, 5)
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.AvgPool2d(kernel_size=2),                 # (B, 64, 2, 2)
    nn.Dropout(0.25),
    nn.Flatten(),                                # (B, 64*2*2)
    nn.Linear(64 * 2 * 2, 128),
    nn.ReLU(),
    nn.Linear(128, 1)
        )
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log('train_loss', loss,on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",          # metric to monitor
    mode="min",                  # minimize val_loss
    save_top_k=1,                # save only the best
    filename="best-checkpoint", # optional name
    verbose=True
)


early_stop_callback = EarlyStopping(
    monitor="val_loss",         # What to monitor
    patience=5,                 # Stop after 3 epochs without improvement
    verbose=True,
    mode="min"                  # Minimize the loss
)

model = RewardCNN()
trainer = pl.Trainer(max_epochs=100, callbacks=[checkpoint_callback,early_stop_callback],log_every_n_steps=1)
trainer.fit(model, train,test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 52.0 K | train
1 | loss_fn | MSELoss    | 0      | train
-----------------------------------------------
52.0 K    Trainable params
0         Non-trainable params
52.0 K    Total params
0.208     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 2500/2500 [00:28<00:00, 87.65it/s, v_num=10, val_loss=0.0185, train_loss=0.0189]

Metric val_loss improved. New best score: 0.019
Epoch 0, global step 2500: 'val_loss' reached 0.01854 (best 0.01854), saving model to '/home/matthieu/Documents/FactorioAI/CNN/lightning_logs/version_10/checkpoints/best-checkpoint.ckpt' as top 1


Epoch 1: 100%|██████████| 2500/2500 [00:28<00:00, 87.64it/s, v_num=10, val_loss=0.0181, train_loss=0.0181] 

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.018
Epoch 1, global step 5000: 'val_loss' reached 0.01806 (best 0.01806), saving model to '/home/matthieu/Documents/FactorioAI/CNN/lightning_logs/version_10/checkpoints/best-checkpoint.ckpt' as top 1


Epoch 2:  30%|██▉       | 749/2500 [00:08<00:19, 90.37it/s, v_num=10, val_loss=0.0181, train_loss=0.0181] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [23]:
model.eval()
with torch.no_grad():
    for i, (x, y_true) in enumerate(test):
        y_pred = model(x)
        y_pred_denorm = scaler_y.inverse_transform(y_pred.cpu().numpy())
        y_true_denorm = scaler_y.inverse_transform(y_true.cpu().numpy())
        for yt, yp in zip(y_true_denorm, y_pred_denorm):
            print(f"True: {yt[0]:.2f}, Predicted: {yp[0]:.2f}")

True: 95.75, Predicted: 47.18
True: -5.50, Predicted: 37.09
True: 45.50, Predicted: 26.59
True: 44.75, Predicted: 41.80
True: 69.50, Predicted: 66.45
True: 144.50, Predicted: 44.90
True: -6.25, Predicted: 37.75
True: -5.25, Predicted: 31.53
True: -6.00, Predicted: 25.15
True: 162.17, Predicted: 61.60
True: 44.75, Predicted: 42.83
True: -6.00, Predicted: 82.27
True: -6.00, Predicted: 22.13
True: 70.00, Predicted: 45.97
True: 44.25, Predicted: 66.21
True: 194.00, Predicted: 70.12
True: -5.25, Predicted: 47.42
True: 44.00, Predicted: 39.15
True: -5.50, Predicted: 24.43
True: 45.50, Predicted: 45.44
True: -5.75, Predicted: 42.60
True: -5.75, Predicted: 53.12
True: -6.25, Predicted: 48.50
True: 95.00, Predicted: 60.69
True: 69.75, Predicted: 32.45
True: 44.50, Predicted: 35.21
True: -5.00, Predicted: 24.49
True: -5.50, Predicted: 33.40
True: -5.75, Predicted: 61.27
True: -5.75, Predicted: 26.45
True: 46.00, Predicted: 27.49
True: -6.25, Predicted: 44.95
True: 44.75, Predicted: 87.11
True: 9